# Capacitated facility location problem

Uma companhia entrega diariamente seus produtos que são produzidos em $m$ fábricas(plantas) para 4 armazens(warehouse). 

Nesse processo de entrega existem dois tipos de custos, o custo associado ao transporte e o custo fixo de uso da fábrica. 

Cada fábrica possui um limite quanto a quantidade de produtos a serem entregues. 

A companhia estuda fechar algumas fábricas para reduzir custos. 

Quais fábricas a companhia deve fechar, com a finalidade de reduzir os custos?

$
\begin{align}
\min \ & \sum_{i \in M} \sum_{j \in N} c_{ij} x_{ij} + \sum_{j \in N} f_j y_j \\
& \sum_{j \in N} x_{ij} = d_i \ \text{ for } i = 1, \ldots, m \\
& \sum_{i \in M} x_{ij} \leq C \cdot y_j \text{ for } j \in N \\
& x_{ij} \geq 0 \text{ for } i \in M, j \in N \\
& y_j \in \{ 0, 1 \} \text{ for } j \in N
\end{align}
$


In [ ]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Dados

# demanda dos armazéns
demand = [15, 18, 14, 20]

# capacidade de produção da cada fábrica
capacity = [20, 22, 17, 19, 18]

# custo fixo de cada fábrica
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# custo de transporte
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# quantidade de fábricas
plants = range(len(capacity))

# quantidade de armazéns
warehouses = range(len(demand))

In [ ]:
# criando modelo
model = gp.Model("cfl")


In [ ]:
# y[p]: y[p] == 1 se a fábrica p é aberta, 0 caso contrário.
y = model.addVars(plants,vtype=GRB.BINARY,obj=fixedCosts,name="y")

In [ ]:
# x[w,p]: quantidade transportada da fábrica p para o armazém p
x = model.addVars(warehouses, plants, obj=transCosts, name="x")

In [ ]:
# função objetivo
model.modelSense = GRB.MINIMIZE
model.update()

In [ ]:
# restrição de produção
model.addConstrs((x.sum('*', p) <= capacity[p] * y[p] for p in plants), "capacidade")
model.update()

In [ ]:
# restrição de demanda
model.addConstrs((x.sum(w,'*') == demand[w] for w in warehouses), "demanda")
model.update()


In [ ]:
# salva o modelo
model.write('cfl.lp')

In [ ]:
# usa o método barrier para resolver o relaxação linear na raiz
model.Params.method = 0

In [ ]:
model.optimize()

In [ ]:
# imprime o valor ótimo
print(f'Custo total: {model.objVal}')


In [ ]:
print('Solução ótima:')
for p in plants:
    if y[p].x > 0.99:
        print(f'* Fábrica {p} é aberto')
        for w in warehouses:
            if x[w, p].x > 0:
                print(f'- Transporte de {x[w, p].x} unidades para o armazém {w}')
    else:
        print(f'* Fábrica {p} é fechada!')